In [3]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tqdm import tqdm

# Constants
CITYSCAPES_DIR = os.path.abspath("../data")
IMAGES_DIR = os.path.join(CITYSCAPES_DIR, "leftImg8bit_trainvaltest", "leftImg8bit", "train")
LABELS_DIR = os.path.join(CITYSCAPES_DIR, "gtFine_trainvaltest", "gtFine", "train")
LIMIT_IMAGES = 1000
LABEL_ID = 7
BATCH_SIZE = 32
TARGET_SIZE = 256
TARGET_HEIGHT = 128

def get_image_label_pairs(images_dir, labels_dir):
    """Retrieve image and label paths."""
    image_label_pairs = []
    for city in os.listdir(images_dir):
        city_folder = os.path.join(images_dir, city)
        label_folder = os.path.join(labels_dir, city)

        # Check if the city folder is indeed a directory
        if not os.path.isdir(city_folder) or not os.path.isdir(label_folder):
            continue

        for image_file in os.listdir(city_folder):
            if image_file.endswith("_leftImg8bit.png"):
                label_file = image_file.replace("_leftImg8bit.png", "_gtFine_labelIds.png")
                label_path = os.path.join(label_folder, label_file)
                if os.path.exists(label_path):
                    image_label_pairs.append((os.path.join(city_folder, image_file), label_path))
                    if len(image_label_pairs) >= LIMIT_IMAGES:
                        return image_label_pairs
    return image_label_pairs

def image_label_generator(image_label_pairs, target_size=(TARGET_HEIGHT, TARGET_SIZE), label_id=LABEL_ID, batch_size=BATCH_SIZE):
    """Generator that yields batches of images and labels."""
    while True:
        X_batch, y_batch = [], []
        for image_path, label_path in tqdm(image_label_pairs):
            image = cv2.imread(image_path)
            label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)

            # Resize images
            X_batch.append(cv2.resize(image, (TARGET_SIZE, TARGET_HEIGHT)))
            y_batch.append((cv2.resize(label, (TARGET_SIZE, TARGET_HEIGHT)) == label_id).astype(np.uint8))

            if len(X_batch) == batch_size:
                yield np.array(X_batch), np.expand_dims(np.array(y_batch), axis=-1)
                X_batch, y_batch = [], []

def build_unet(input_shape):
    """Build U-Net model."""
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(4, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.SeparableConv2D(8, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    skip1 = x
    x = layers.MaxPooling2D(2)(x)

    x = layers.SeparableConv2D(16, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    skip2 = x
    x = layers.MaxPooling2D(2)(x)

    x = layers.SeparableConv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.SeparableConv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Concatenate()([x, skip2])

    x = layers.SeparableConv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Concatenate()([x, skip1])

    x = layers.SeparableConv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, (1, 1), padding='same', activation='relu')(x)
    outputs = layers.Conv2D(1, (1, 1), padding='same', activation='sigmoid')(x)

    model = models.Model(inputs, outputs)
    model.compile(optimizer=optimizers.Adam(learning_rate=0.01),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

def train_model(train_generator, val_generator, steps_per_epoch , validation_steps, epochs=20):
    """Train the U-Net model using the provided generators."""
    input_shape = (TARGET_HEIGHT, TARGET_SIZE, 3)
    model = build_unet(input_shape)
    model.summary()

    # Callbacks for model checkpointing and early stopping
    checkpoint_cb = callbacks.ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_loss", mode="min")
    early_stopping_cb = callbacks.EarlyStopping(patience=3, restore_best_weights=True, monitor="val_loss", mode="min")

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=validation_steps,
        callbacks=[checkpoint_cb, early_stopping_cb],
        verbose=1
    )
    return model, history

In [ ]:
# Example usage
EPOCHS = 10
STEPS_PER_EPOCH = 100
VALIDATION_STEPS = 10

# Prepare image-label pairs and create generators
image_label_pairs = get_image_label_pairs(IMAGES_DIR, LABELS_DIR)
train_generator = image_label_generator(image_label_pairs)
val_generator = image_label_generator(image_label_pairs)  # Replace with actual validation pairs

# Train the model
model, history = train_model(train_generator, val_generator, STEPS_PER_EPOCH, VALIDATION_STEPS, EPOCHS)

# Summarize the model
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 256,  │        112 │ input_layer[0][0] │
│                     │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 256,  │         16 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 4)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 128, 256,  │         76 │ batch_normalizat… │
│ (SeparableConv2D)   │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 256,  │         32 │ separable_conv2d… │
│ (BatchNormalizatio… │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 128,   │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 64, 128,   │        216 │ max_pooling2d[0]… │
│ (SeparableConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128,   │         64 │ separable_conv2d… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 64,    │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_2  │ (None, 32, 64,    │        688 │ max_pooling2d_1[… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_3  │ (None, 32, 64,    │      1,344 │ batch_normalizat… │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 32, 64,    │        128 │ separable_conv2d… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d       │ (None, 64, 128,   │          0 │ batch_normalizat… │
│ (UpSampling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64, 128,   │          0 │ up_sampling2d[0]… │
│ (Concatenate)       │ 48)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_4  │ (None, 64, 128,   │      2,000 │ concatenate[0][0] │
│ (SeparableConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 128,   │        128 │ separable_conv2d

 Total params: 11,085 (43.30 KB)

 Trainable params: 10,773 (42.08 KB)

 Non-trainable params: 312 (1.22 KB)

...

 95%|█████████▌| 950/1000 [02:23<00:07,  6.72it/s]

 91/100 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step - accuracy: 0.8354 - loss: 0.3539

 98%|█████████▊| 982/1000 [02:28<00:02,  6.95it/s]

 92/100 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.8357 - loss: 0.3534

 98%|█████████▊| 984/1000 [02:28<00:02,  6.79it/s]